In [ ]:
# Part 3: Bounding Box Tracking with Optical Flow

import zipfile # For extracting 'data.zip'
import os # For image file handling
import cv2 # for Optical Flow
import numpy as np # For calculations
import matplotlib.pyplot as plt  # For visualisations

# In order for extraction below to work, need to first upload 'data.zip' to Files
# Extracting 'data.zip'
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
# Output folder to store every 5th frame
bbof_results = 'BoundingBox_Results_OpticalFlow/'
os.makedirs(bbof_results, exist_ok=True) # creating folder if it does not yet exist

# Initial bounding box (x, y, w, h)
og_box = (20, 353, 322, 215)
box_x, box_y, box_w, box_h = og_box
prev_box = og_box # initialising the prev_box variable here since it is utilised in for loop

# Drawing the original bounding box on frame_0
frame0_path = os.path.join(f'data/frame_0.jpg')
frame0_BGR = cv2.imread(frame0_path)
cv2.rectangle(frame0_BGR, (box_x, box_y), (box_x + box_w, box_y + box_h), (0, 255, 0), 2)

# Displaying the updated box
plt.figure(figsize=(12, 6))
plt.imshow(cv2.cvtColor(frame0_BGR, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title(f"Bounding Box Tracking with SIFT: frame_0")
plt.show()

# Going through frames 1-69, using the previous frame to update the box for the next frame
for i in range(1, 70):
    f1_path = os.path.join(f'data/frame_{i-1}.jpg')
    f2_path = os.path.join(f'data/frame_{i}.jpg')

    # Loading the frames
    frame1_BGR = cv2.imread(f1_path)
    frame2_BGR = cv2.imread(f2_path)

    # Converting the frame to grayscale for Optical Flow
    frame1_gray = cv2.cvtColor(frame1_BGR, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.cvtColor(frame2_BGR, cv2.COLOR_BGR2GRAY)

    # If either frame is not loaded, raise an error
    if frame1_gray is None or frame2_gray is None:
      raise ValueError("Error loading frame. Check that the file path exists.")

    # Step 1: Creating a grid of points (dense sampling)
    step_size = 20
    f1_h, f1_w = frame1_gray.shape
    grid_x, grid_y = np.meshgrid(np.arange(0, f1_w, step_size), np.arange(0, f1_h, step_size))
    points = np.vstack((grid_x.ravel(), grid_y.ravel())).T
    points_reshaped = np.array(points, dtype=np.float32).reshape(-1,1,2)

    # Step 2: Calculating optical flow for each point using Lucas-Kanade method
    lk_params = dict(winSize=(15, 15), maxLevel=4)
    flow_points, status, err = cv2.calcOpticalFlowPyrLK(frame1_gray, frame2_gray, points_reshaped, None, **lk_params)
    old_points = points_reshaped[status==1]
    new_points = flow_points[status==1]

    # Step 3: Filtering to get just the points inside the bounding box
    box_x, box_y, box_w, box_h = prev_box
    # old_points[:, 0] - retrieves x-coordinates of all points, old_points[:, 1] retrieves y-coordinates
    inside_bb_mask = (
        (box_x <= old_points[:, 0]) & (old_points[:, 0] <= box_x + box_w) &
        (box_y <= old_points[:, 1]) & (old_points[:, 1] <= box_y + box_h)
    )
    old_inside_bb = old_points[inside_bb_mask] # Extracting the points within the box from the previous frame
    new_inside_bb = new_points[inside_bb_mask] # Extracting those points' updated positions in the current frame

    # Step 4: Updating the bounding box position based on the median displacement
    disp_vectors = new_inside_bb - old_inside_bb
    # Calculating the median displacement
    median_x_disp = np.median(disp_vectors[:, 0])
    median_y_disp = np.median(disp_vectors[:, 1])

    # Using the median displacement to update the centre of the box
    # Scaling factor of 1.1 applied because without it, box notably lags behind car in last ten frames
    x_centre = (box_x + (box_w / 2)) + (median_x_disp * 1.1)
    y_centre = (box_y + (box_h / 2)) + (median_y_disp * 1.1)

    # Adjusting the box's x and y based on the newly calculated centre
    box_x = int(x_centre - (box_w / 2))
    box_y = int(y_centre - (box_h / 2))
    # Retaining the width and height so the box doesn't grow or shrink
    prev_box = (box_x, box_y, box_w, box_h) # Saving the updated box for the next iteration

    # Drawing the updated bounding box on frame2_BGR
    cv2.rectangle(frame2_BGR, (box_x, box_y), (box_x + box_w, box_y + box_h), (0, 255, 0), 2)

    # Visualizing the second frame with the updated bounding box
    plt.figure(figsize=(12, 6))
    plt.imshow(cv2.cvtColor(frame2_BGR, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f"Bounding Box Tracking with Optical Flow: frame_{i}")
    plt.show()

    # Saving every fifth frame's resulting image
    if i % 5 == 0:
      filename = os.path.join(bbof_results, f'frame_{i}.jpg')
      cv2.imwrite(filename, frame2_BGR)

## The cell below is only needed if running the code in Google Colab rather than locally on Jupyter Notebook.

In [ ]:
# Downloading the saved 'BoundingBox_Results_OpticalFlow' from Google Colab to my local machine

from google.colab import files
import shutil

# Converting the file into a zip file
shutil.make_archive('BoundingBox_Results_OpticalFlow', 'zip', 'BoundingBox_Results_OpticalFlow')

# Downloading the zip file to my local machine
files.download('BoundingBox_Results_OpticalFlow.zip')